In [1]:
@file:DependsOn("com.google.adk:google-adk:0.3.0")
@file:DependsOn("com.google.adk:google-adk-dev:0.2.0")
@file:DependsOn("com.google.genai:google-genai:1.23.0")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-rx3:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-reactive:1.8.1")

In [2]:
import com.google.adk.agents.LlmAgent
import com.google.adk.agents.SequentialAgent
import com.google.adk.agents.ParallelAgent
import com.google.adk.agents.LoopAgent

import com.google.adk.models.Gemini
import com.google.genai.Client

import com.google.adk.runner.InMemoryRunner
import com.google.adk.sessions.Session
import com.google.adk.sessions.InMemorySessionService
import com.google.adk.memory.InMemoryMemoryService
import com.google.adk.events.Event
import com.google.adk.tools.FunctionTool
import com.google.adk.tools.GoogleSearchTool

import com.google.genai.types.Content
import com.google.genai.types.Part

import java.util.UUID
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.toList
import kotlinx.coroutines.reactive.asFlow

import io.reactivex.rxjava3.core.Flowable;

---

## 🤖 Section 2: Your first AI Agent with ADK

### 🤔 2.1 What is an AI Agent?

You've probably used an LLM like Gemini before, where you give it a prompt and it gives you a text response.

`Prompt -> LLM -> Text`

An AI Agent takes this one step further. An agent can think, take actions, and observe the results of those actions to give you a better answer.

`Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer`

In this notebook, we'll build an agent that can take the action of searching Google. Let's see the difference!

In [3]:
val apiKey = System.getenv("GOOGLE_API_KEY")
if (apiKey == null) {
    throw IllegalStateException("Please set the GOOGLE_API_KEY environment variable.")
}

In [4]:
val geminiClient = Client.builder().apiKey(apiKey).build()

In [5]:
val geminiModel = Gemini(
    "gemini-2.5-flash-lite", // Your model name
    geminiClient
)

### 2.2 Define your agent

        Now, let's build our agent. We'll configure an `Agent` by setting its key properties, which tell it what to do and how to operate.

To learn more, check out the documentation related to [agents in ADK](https://google.github.io/adk-docs/agents/).

These are the main properties we'll set:

        - **name** and **description**: A simple name and description to identify our agent.
        - **model**: The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite".
- **instruction**: The agent's guiding prompt. This tells the agent what its goal is and how to behave.
        - **tools**: A list of [tools](https://google.github.io/adk-docs/tools/) that the agent can use. To start, we'll give it the `google_search` tool, which lets it find up-to-date information online.

In [6]:
val rootAgent = LlmAgent.builder()
    .name("helpful_assistant")
    .model(geminiModel)
    .description("A simple agent that can answer general questions")
    .instruction("""You are a helpful assistant. Use Google Search for current info or if unsure""")
    .tools(GoogleSearchTool.INSTANCE)
    .build()

println("✅ Root agent defined.")

✅ Root agent defined.


In [7]:
var runner: InMemoryRunner = InMemoryRunner(rootAgent, "Assistant Flow")

In [8]:
var prompt = "What is Agent Development Kit from Google? What languages is the SDK available in"

var userMsg: Content? = Content.fromParts(Part.fromText(prompt))

In [9]:
var session: Session? = runner
    .sessionService()
    .createSession("Assistant Flow", "user")
    .blockingGet()

### 2.3 Run your agent

        Now it's time to bring your agent to life and send it a query. To do this, you need a [`Runner`](https://google.github.io/adk-docs/runtime/), which is the central component within ADK that acts as the orchestrator. It manages the conversation, sends our messages to the agent, and handles its responses.

**a. Create an `InMemoryRunner` and tell it to use our `root_agent`:**

In [15]:
println("Agent response ->")
runBlocking {
    var events = runner.runAsync("user", session.id(), userMsg)
        .asFlow()
        .toList()
    events.forEach {
        event -> println(event.stringifyContent())
    }
}

Agent response ->
Kraftwerk has two upcoming concerts in Berlin in December 2025. They are scheduled to perform on **December 9 and December 10, 2025**, both at the Uber Eats Music Hall. These performances are part of their "Multimedia Tour 2025."


You can see a summary of ADK and its available languages in the response.

### 2.4 How does it work?

The agent performed a Google Search to get the latest information about ADK, and it knew to use this tool because:

1. The agent inspects and is aware of which tools it has available to use.
2. The agent's instructions specify the use of the search tool to get current information or if it is unsure of an answer.

The best way to see the full, detailed trace of the agent's thoughts and actions is in the **ADK web UI**, which we'll set up later in this notebook.

And we'll cover more detailed workflows for logging and observability later in the course.

In [11]:
prompt = "What did I just ask you about"
userMsg = Content.fromParts(Part.fromText(prompt))
events = runner.runAsync("user", session.id(), userMsg)
println("Agent response ->")
events.blockingForEach { event -> println(event.stringifyContent()) }

Agent response ->
You asked me about the Agent Development Kit (ADK) from Google and the programming languages in which the SDK is available.


In [12]:
prompt = "Can you tell me when is the next Kraftwerk concert in Berlin?"
userMsg = Content.fromParts(Part.fromText(prompt))
events = runner.runAsync("user", session.id(), userMsg)
println("Agent response ->")
events.blockingForEach { event -> println(event.stringifyContent()) }

Agent response ->
Kraftwerk has two upcoming concerts in Berlin on **December 9 and 10, 2025**, both at the Uber Eats Music Hall. Tickets for these shows are available, with some already listed as "few tickets left" or "about to be sold out."


---

## 💻 Section 3: Try the ADK Web Interface

### Overview

ADK includes a built-in web interface for interactively chatting with, testing, and debugging your agents.

<img width="1200" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/adk-web-ui.gif" alt="ADK Web UI" />

To use the ADK web UI, you'll need to create an agent with Python files using the `adk create` command.

Run the command below to generate a `sample-agent` folder that contains all the necessary files, including `agent.py` for your code, an `.env` file with your API key pre-configured, and an `__init__.py` file:

---

## ✅ Congratulations!

You've built and run your first agent with ADK! You've just seen the core concept of agent development in action.

The big takeaway is that your agent didn't just *respond*—it **reasoned** that it needed more information and then **acted** by using a tool. This ability to take action is the foundation of all agent-based AI.

**ℹ️ Note: No submission required!**

        This notebook is for your hands-on practice and learning only. You **do not** need to submit it anywhere to complete the course.

### 📚 Learn More

        Refer to the following documentation to learn more:

- [ADK Documentation](https://google.github.io/adk-docs/)
- [ADK Quickstart for Python](https://google.github.io/adk-docs/get-started/python/)
- [ADK Agents Overview](https://google.github.io/adk-docs/agents/)
- [ADK Tools Overview](https://google.github.io/adk-docs/tools/)

### 🎯 Next Steps

        Ready for the next challenge? Continue to the next notebook to learn how to **architect multi-agent systems.**